### In this lab, the objective is to identify the customers who were active in both May and June, and how did their activity differ between months. To achieve this, follow these steps:

1. Establish a connection between Python and the Sakila database.


In [ ]:
!pip install sqlalchemy
!pip install sql
!pip install mysql
!pip install pymysql
!pip install mysql-connector-python

In [7]:
#import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pymysql
from sqlalchemy import create_engine
import getpass  # To get the password without showing the input
password = getpass.getpass()

In [11]:
user = "root"
password = "1234abcd"
host = "127.0.0.1"
port = "3306"
db = "sakila"
#create engine
engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}:{port}/{db}')

In [23]:
#test
query1 = "SELECT * FROM rental"
df_read1 = pd.read_sql(query1, con=engine)
df_read1.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53


### Challenge 2:

Write a Python function called rentals_month that retrieves rental data for a given month and year (passed as parameters) from the Sakila database as a Pandas DataFrame. The function should take in three parameters:

1. engine: an object representing the database connection engine to be used to establish a connection to the Sakila database.
2. month: an integer representing the month for which rental data is to be retrieved.
3. year: an integer representing the year for which rental data is to be retrieved.
The function should execute a SQL query to retrieve the rental data for the specified month and year from the rental table in the Sakila database, and return it as a pandas DataFrame.

In [22]:
def rentals_month(engine, month, year):
    # SQL query to retrieve rental data for a specific month and year
    query1 = f"""
    SELECT *
    FROM sakila.rental
    WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year};
    """
    
    # Execute the query and read the results into a pandas DataFrame
    df_read1 = pd.read_sql(query1, con=engine)
    
    # Return the DataFrame
    return df_read1

# Call function as follows (e.g., for March 2005):
df_march_2005 = rentals_month(engine, 5, 2005)
df_march_2005.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53


### Challenge 3:

Develop a Python function called rental_count_month that takes the DataFrame provided by rentals_month as input along with the month and year and returns a new DataFrame containing the number of rentals made by each customer_id during the selected month and year.

The function should also include the month and year as parameters and use them to name the new column according to the month and year, for example, if the input month is 05 and the year is 2005, the column name should be "rentals_05_2005".

Hint: Consider making use of pandas groupby()


In [25]:
def rental_count_month(engine, month, year):
    # Dynamically build the SQL query string
    query = f"""
    SELECT customer_id, COUNT(*) AS rentals_{month:02d}_{year}
    FROM rental
    WHERE YEAR(rental_date) = {year} AND MONTH(rental_date) = {month}
    GROUP BY customer_id;
    """
    
    # Execute the query and return the results as a DataFrame
    df = pd.read_sql(query, con=engine)
    
    return df


result_df = rental_count_month(engine, 5, 2005)
result_df.head()

,customer_id,rentals_05_2005
0,1,2
1,2,1
2,3,2
3,5,3
4,6,3


### Challenge 4:

Create a Python function called compare_rentals that takes two DataFrames as input containing the number of rentals made by each customer in different months and years. The function should return a combined DataFrame with a new 'difference' column, which is the difference between the number of rentals in the two months.

In [29]:
def compare_rentals(engine, month1, year1, month2, year2):
    # Define the SQL queries for each month
    query1 = f"""
    SELECT customer_id, COUNT(*) AS rentals_{month1:02d}_{year1}
    FROM rental
    WHERE YEAR(rental_date) = {year1} AND MONTH(rental_date) = {month1}
    GROUP BY customer_id;
    """
    
    query2 = f"""
    SELECT customer_id, COUNT(*) AS rentals_{month2:02d}_{year2}
    FROM rental
    WHERE YEAR(rental_date) = {year2} AND MONTH(rental_date) = {month2}
    GROUP BY customer_id;
    """
    
    # Execute the SQL queries and retrieve the data as pandas DataFrames
    df1 = pd.read_sql(query1, con=engine)
    df2 = pd.read_sql(query2, con=engine)
    
    # Merge the two DataFrames on 'customer_id'
    df_combined = pd.merge(df1, df2, on='customer_id', how='outer')
    
    # Fill NaN values with 0, assuming no rentals means count as 0
    df_combined.fillna(0, inplace=True)
    
    # Calculate the difference between the number of rentals in the two months
    col_name1 = f'rentals_{month1:02d}_{year1}'
    col_name2 = f'rentals_{month2:02d}_{year2}'
    df_combined['difference'] = df_combined[col_name1] - df_combined[col_name2]
    
    return df_combined

#Call function
result_df = compare_rentals(engine, 5, 2005, 6, 2005)
result_df.head()

,customer_id,rentals_05_2005,rentals_06_2005,difference
0,1,2.0,7.0,-5.0
1,2,1.0,1.0,0.0
2,3,2.0,4.0,-2.0
3,4,0.0,6.0,-6.0
4,5,3.0,5.0,-2.0
